In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import sys

curruser = os.environ.get('USER')
sys.path.insert(0, './src/')
# sys.path.insert(0, '/home/{}/notebooks/support_library/'.format(curruser)) 
sys.path.insert(0, '/home/{}/python36-libs/lib/python3.6/site-packages/'.format(curruser))
from pathlib import Path
import re
import csv
import itertools
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook
# import utils
from LSTMPreProc import preprocIO
# from LSTMPostProc import LSTM_pred
from Bio import SeqIO
from sklearn.preprocessing import LabelEncoder

In [3]:
dnaSegPath = ('./data/dnaseg/mart_export.txt')

In [4]:
prep = preprocIO()

### IOStreaming: Fetching data from FASTA format

In [5]:
seqArr = []
for seq in tqdm(prep.streamFastaIO(dnaSegPath)):
     seqArr+=[seq]

102020it [00:04, 23548.59it/s]


In [6]:
chars = sorted(list(set(seqArr[0])))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [7]:
char_indices

{'A': 0, 'C': 1, 'G': 2, 'T': 3}

In [11]:
csvpath = Path.joinpath(Path(os.getcwd()),'data/dnaseg/csv','DnaSeg4Train.csv')

In [ ]:
# finalarr = []
# headers = ['seq']
# for item in tqdm(seqArr):
#     with open(str(csvpath),'a', encoding='utf8') as outcsv: 
#         writer = csv.writer(outcsv,delimiter=';', quotechar='"', lineterminator='\n')
#         file_is_empty = os.stat(str(csvpath)).st_size ==0
#         if file_is_empty:
#             writer.writerow(headers)
#         writer.writerow([item])


## Unidirectional RandomSampling

In [7]:
import random
random.seed(42)

In [8]:
np.random.shuffle(seqArr)

In [9]:
finalarr = []
headers = ['seq', 'label']
csvpath = Path.joinpath(Path(os.getcwd()),'data/dnaseg/csv','DnaSeg4TrainwLabels_wRandL.csv')
for item in tqdm(seqArr):
    chunks = prep.RandomSamplingGen(item, 
                                    randsize_center=150,
                                    min_left_len=30, max_left_len=60,
                                    min_right_len=30, max_right_len=60,)
    out = [(chunk[0], char_indices[chunk[1]]) for chunk in chunks]
    with open(str(csvpath),'a', encoding='utf8') as outcsv: 
        writer = csv.writer(outcsv,delimiter=';', quotechar='"', lineterminator='\n')
        file_is_empty = os.stat(str(csvpath)).st_size ==0
        if file_is_empty:
            writer.writerow(headers)
        for row in out:
            writer.writerow(row)

100%|██████████| 102020/102020 [20:13<00:00, 84.05it/s]


In [11]:
csvpath = Path.joinpath(Path(os.getcwd()),'data/dnaseg/csv','DnaSeg4TrainwLabels_wRandL.csv')
df = pd.read_csv(csvpath, sep=';')

In [12]:
df.shape

(14267163, 2)

In [13]:
df.drop_duplicates(subset=['seq'],inplace=True)

In [14]:
df.shape

(14159075, 2)

In [15]:
csvpath = Path.joinpath(Path(os.getcwd()),'data/dnaseg/csv','DnaSeg4TrainwLabelsSampled_wRandL.csv')
df_frac = df.groupby('label', group_keys=False).apply(lambda x: x.sample(n=int(np.rint(0.2*len(x))), random_state=42, axis=0))
df_frac.shape

(2831814, 2)

In [16]:
df_frac.to_csv(csvpath,index=False, encoding='utf8',sep=';')

In [17]:
indx = np.array(df_frac.index.tolist())

In [18]:
oobdf = df[~df.index.isin(indx)]
oobdf_frac = oobdf.groupby('label', group_keys=False).apply(lambda x: x.sample(n=int(np.rint(0.1*len(x))), random_state=42, axis=0))

In [19]:
csvpath = Path.joinpath(Path(os.getcwd()),'data/dnaseg/csv','DnaSeg4ValwLabelsSampled_wRandL.csv')
oobdf_frac.to_csv(csvpath,index=False, encoding='utf8',sep=';')

In [13]:
# csvpath = Path.joinpath(Path(os.getcwd()),'data/dnaseg/csv','DnaSeg4TrainwLabelsSampled_wRandL.csv')
# df = pd.read_csv(csvpath, sep=';')

## Convert sentences to bag-of-CharTriGrams

In [13]:
def gen_trigrams():
  """
      Generates all trigrams for characters from `trigram_chars`
  """
  trigram_chars="ACGT"
  t3=[''.join(x) for x in itertools.product(trigram_chars,repeat=3)] #len(words)>=3
  t2_start=['#'+''.join(x) for x in itertools.product(trigram_chars,repeat=2)] #len(words)==2
  t2_end=[''.join(x)+'#' for x in itertools.product(trigram_chars,repeat=2)] #len(words)==2
  t1=['#'+''.join(x)+'#' for x in itertools.product(trigram_chars)] #len(words)==1
  trigrams=t3+t2_start+t2_end+t1
  vocab_size=len(trigrams)
  trigram_map=dict(zip(trigrams,range(1,vocab_size+1))) # trigram to index mapping, indices starting from 1
  return trigram_map

In [14]:
def sentences_to_bag_of_trigrams(sentences):
    """
      Converts a sentence to bag-of-trigrams
      `sentences`: list of strings
      `trigram_BOW`: return value, (len(sentences),len(trigram_map)) size array
    """
    trigram_map=gen_trigrams()
    trigram_BOW=np.zeros((len(sentences),len(trigram_map))) # one row for each sentence
    filter_pat=r'[\!"#&\(\)\*\+,-\./:;<=>\?\[\\\]\^_`\{\|\}~\t\n]' # characters to filter out from the input
    for j,sent in enumerate(sentences):
      sent=re.sub(fiter_pat, '', sent).lower() # filter out special characters from input
      sent=re.sub(r"(\s)\s+", r"\1", sent) # reduce multiple whitespaces to single whitespace
      words=sent.split(' ')
      indices=collections.defaultdict(int)
      for word in words:
          word='#'+word+'#'
          #print(word)
          for k in range(len(word)-2): # generate all trigrams for word `word` and update `indices`
              trig=word[k:k+3]
              idx=trigram_map.get(trig, 0)
              #print(trig,idx)
              indices[idx]=indices[idx]+1     
      for key,val in indices.items(): #covert `indices` dict to np array
          trigram_BOW[j,key]=val
    return trigram_BOW

In [15]:
trigram_map=gen_trigrams()

In [16]:
trigram_map

{'AAA': 1,
 'AAC': 2,
 'AAG': 3,
 'AAT': 4,
 'ACA': 5,
 'ACC': 6,
 'ACG': 7,
 'ACT': 8,
 'AGA': 9,
 'AGC': 10,
 'AGG': 11,
 'AGT': 12,
 'ATA': 13,
 'ATC': 14,
 'ATG': 15,
 'ATT': 16,
 'CAA': 17,
 'CAC': 18,
 'CAG': 19,
 'CAT': 20,
 'CCA': 21,
 'CCC': 22,
 'CCG': 23,
 'CCT': 24,
 'CGA': 25,
 'CGC': 26,
 'CGG': 27,
 'CGT': 28,
 'CTA': 29,
 'CTC': 30,
 'CTG': 31,
 'CTT': 32,
 'GAA': 33,
 'GAC': 34,
 'GAG': 35,
 'GAT': 36,
 'GCA': 37,
 'GCC': 38,
 'GCG': 39,
 'GCT': 40,
 'GGA': 41,
 'GGC': 42,
 'GGG': 43,
 'GGT': 44,
 'GTA': 45,
 'GTC': 46,
 'GTG': 47,
 'GTT': 48,
 'TAA': 49,
 'TAC': 50,
 'TAG': 51,
 'TAT': 52,
 'TCA': 53,
 'TCC': 54,
 'TCG': 55,
 'TCT': 56,
 'TGA': 57,
 'TGC': 58,
 'TGG': 59,
 'TGT': 60,
 'TTA': 61,
 'TTC': 62,
 'TTG': 63,
 'TTT': 64,
 '#AA': 65,
 '#AC': 66,
 '#AG': 67,
 '#AT': 68,
 '#CA': 69,
 '#CC': 70,
 '#CG': 71,
 '#CT': 72,
 '#GA': 73,
 '#GC': 74,
 '#GG': 75,
 '#GT': 76,
 '#TA': 77,
 '#TC': 78,
 '#TG': 79,
 '#TT': 80,
 'AA#': 81,
 'AC#': 82,
 'AG#': 83,
 'AT#': 84,
 

In [ ]:
trigram_BOW = sentences_to_bag_of_trigrams(seqArr)

### Cut the corpus into BiDirectional chunks of 30 characters (shift by the left and right from the next character to predict), spacing the sequences by 5 characters.

In [6]:
SEQUENCE_LENGTH = 30
step = 5
sentences = []

In [10]:
sentences, next_chars = prep.genSeqMoveRightChar(seqArr, SEQUENCE_LENGTH, step)

100%|██████████| 102020/102020 [00:11<00:00, 8720.30it/s]


In [11]:
sentences, next_chars_indx = prep.genSeqMoveRight(seqArr, SEQUENCE_LENGTH, step, char_indices)

100%|██████████| 102020/102020 [00:11<00:00, 8570.95it/s]


In [ ]:
# df = pd.DataFrame({'seq':sentences,'label':next_chars_indx})

In [12]:
df = pd.DataFrame({'seq':sentences,'label':next_chars})

### Sampling some fraction from generated sequences

In [13]:
df_frac = df.groupby('label', group_keys=False).apply(lambda x: x.sample(n=int(np.rint(0.4*len(x))), random_state=42, axis=0))
# df_frac.reset_index(inplace=True)


In [14]:
df_frac.label.value_counts()

G    2451616
A    2436500
C    2422670
T    2045139
Name: label, dtype: int64

### Save dataframe for Train/Test

In [15]:
csvpath = Path.joinpath(Path(os.getcwd()),'data/dnaseg/csv','DnaSeg4TrainCharLabel.csv')
df_frac.to_csv(csvpath,index=False, encoding='utf8',sep=';')

In [29]:
# csvpath = Path.joinpath(Path(os.getcwd()),'data/dnaseg/csv','DnaSeg4TrainwLabels.csv')
df_frac.to_csv(csvpath,index=False, encoding='utf8',sep=';')

### Save dataframe for OOB validation

In [16]:
indx = np.array(df_frac.index.tolist())

In [17]:
oobdf = df[~df.index.isin(indx)]
oobdf_frac = oobdf.groupby('label', group_keys=False).apply(lambda x: x.sample(n=int(np.rint(0.1*len(x))), random_state=42, axis=0))

In [33]:
oobdf_frac.label.value_counts()

2    367742
0    365475
1    363400
3    306771
Name: label, dtype: int64

In [18]:
csvpath = Path.joinpath(Path(os.getcwd()),'data/dnaseg/csv','DnaSeg4ValCharLabel.csv')
oobdf_frac.to_csv(csvpath,index=False, encoding='utf8',sep=';')

In [34]:
# csvpath = Path.joinpath(Path(os.getcwd()),'data/dnaseg/csv','DnaSeg4ValwLabels.csv')
# oobdf_frac.to_csv(csvpath,index=False, encoding='utf8',sep=';')

### Generate Bidirectional chunked sequences and transform omitted char into its label encoding 

In [ ]:
lenc = LabelEncoder()
lenc.fit_transform(np.array(['A','T','C','G']))

In [ ]:
finalarr = []
headers = ['seq']
for item in tqdm(seqArr):
    
    chunkesRight = prep.getChunkesFromSeqToRight(item, lenc = lenc)
    chunkesLeft  = prep.getChunkesFromSeqToLeft(item,  lenc = lenc)
    res = chunkesRight+chunkesLeft
    finalarr+=res

    with open(str(csvpath),'a', encoding='utf8') as outcsv: 
        writer = csv.writer(outcsv,delimiter=';', quotechar='"', lineterminator='\n')
        file_is_empty = os.stat(str(csvpath)).st_size ==0
        if file_is_empty:
            writer.writerow(headers)
        for item in finalarr:
            writer.writerow(item)
 